## 🧠 Ensemble de Ensembles — *Model Stacking*

### 🔹 Ensemble con diferentes tipos de clasificadores
Se combinan **diferentes tipos de modelos** (por ejemplo, *Regresión Logística*, *Árboles de Decisión*, *Random Forest*, etc.) entrenados con el **mismo conjunto de datos**.

Los resultados se combinan mediante:
- **Votación mayoritaria** → para problemas de **clasificación**.  
- **Promedio (media)** → para problemas de **regresión**.

---

### 🔹 Ensemble con un solo tipo de clasificador
- Se generan **muestras bootstrap** del conjunto de entrenamiento.  
- Cada muestra se usa para entrenar un modelo (por ejemplo, *árboles de decisión* o *random forest*).  
- Todos los resultados se **combinan** para formar un *ensemble*.  

📈 Este método es ideal para **modelos muy flexibles**, propensos al **sobreajuste (overfitting)** o **alta varianza**.

---

### 🔹 Métodos de combinación

#### 🗳️ Clasificación
- Se selecciona el **resultado con más votos** (*mode*).

#### 📉 Regresión
- Se calcula el **promedio (mean)** de las predicciones.

---

### 🔹 Aplicación de *Meta-Classifiers* (Clasificadores de nivel superior)

#### 1️⃣ Sobre resultados binarios
- Los modelos base producen salidas binarias (*0 / 1*).  
- Un **meta-clasificador** se entrena sobre estas salidas para realizar la predicción final.

#### 2️⃣ Sobre probabilidades
- Los modelos base devuelven **probabilidades** en lugar de clases.  
- El meta-clasificador se aplica sobre esas probabilidades para combinar la información de manera más precisa.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv("data//WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.pop('EmployeeNumber')
df.pop('Over18')
df.pop('StandardHours')
df.pop('EmployeeCount')
y = df['Attrition']
X = df
X.pop('Attrition')
from sklearn import preprocessing
le = preprocessing.LabelBinarizer()
y = le.fit_transform(y)
ind_BusinessTravel = pd.get_dummies(df['BusinessTravel'], prefix='BusinessTravel')
ind_Department = pd.get_dummies(df['Department'], prefix='Department')
ind_EducationField = pd.get_dummies(df['EducationField'], prefix='EducationField')
ind_Gender = pd.get_dummies(df['Gender'], prefix='Gender')
ind_JobRole = pd.get_dummies(df['JobRole'], prefix='JobRole')
ind_MaritalStatus = pd.get_dummies(df['MaritalStatus'], prefix='MaritalStatus')
ind_OverTime = pd.get_dummies(df['OverTime'], prefix='OverTime')
df1 = pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, 
                 ind_JobRole, ind_MaritalStatus, ind_OverTime])
df1 = pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, 
                 ind_JobRole, ind_MaritalStatus, ind_OverTime, df.select_dtypes(['int64'])], axis=1)
df1.dropna(inplace=True)
df1.shape

(1470, 51)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, y)

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                        lb.transform(clf.predict(X_train)))))

        #cv_res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(cv_res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(cv_res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))      
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))

## Modelo 1 -> Arbol de decision

In [3]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

print_score(tree_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       920
           1       1.00      1.00      1.00       182

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[920   0]
 [  0 182]]

ROC AUC: 1.0000



In [4]:
print_score(tree_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8125

Classification Report: 
               precision    recall  f1-score   support

           0       0.90      0.88      0.89       313
           1       0.39      0.44      0.41        55

    accuracy                           0.81       368
   macro avg       0.64      0.66      0.65       368
weighted avg       0.82      0.81      0.82       368


Confusion Matrix: 
 [[275  38]
 [ 31  24]]

ROC AUC: 0.6575



---
## Modelo 2 -> Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train.ravel())

print_score(rf_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       920
           1       1.00      1.00      1.00       182

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[920   0]
 [  0 182]]

ROC AUC: 1.0000



In [6]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8614

Classification Report: 
               precision    recall  f1-score   support

           0       0.86      1.00      0.92       313
           1       0.83      0.09      0.16        55

    accuracy                           0.86       368
   macro avg       0.85      0.54      0.54       368
weighted avg       0.86      0.86      0.81       368


Confusion Matrix: 
 [[312   1]
 [ 50   5]]

ROC AUC: 0.5439



In [7]:
en_en = pd.DataFrame()
tree_clf.predict_proba(X_train)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], shape=(1102, 2))

In [8]:
tree_clf.predict_proba?

Signature: tree_clf.predict_proba(X, check_input=True)
Docstring:
Predict class probabilities of the input samples X.

The predicted class probability is the fraction of samples of the same
class in a leaf.

Parameters
----------
X : {array-like, sparse matrix} of shape (n_samples, n_features)
    The input samples. Internally, it will be converted to
    ``dtype=np.float32`` and if a sparse matrix is provided
    to a sparse ``csr_matrix``.

check_input : bool, default=True
    Allow to bypass several input checking.
    Don't use this parameter unless you know what you're doing.

Returns
-------
proba : ndarray of shape (n_samples, n_classes) or list of n_outputs             such arrays if n_outputs > 1
    The class probabilities of the input samples. The order of the
    classes corresponds to that in the attribute :term:`classes_`.
File:      c:\users\pcvip\desktop\cursosudemy\completemachinelearning\.venv\lib\site-packages\sklearn\tree\_classes.py
Type:      method

In [9]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]
en_en['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_train))[1]
col_name = en_en.columns
en_en = pd.concat([en_en, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

en_en.head()

,tree_clf,rf_clf,0
0,0.0,0.00,0
1,0.0,0.01,0
2,0.0,0.07,0
3,1.0,0.84,1
4,0.0,0.03,0


In [10]:
tmp = list(col_name)
tmp.append('ind')
en_en.columns = tmp

en_en.head()

,tree_clf,rf_clf,ind
0,0.0,0.00,0
1,0.0,0.01,0
2,0.0,0.07,0
3,1.0,0.84,1
4,0.0,0.03,0


---
## Meta Classifier

In [11]:
from sklearn.linear_model import LogisticRegression

m_clf = LogisticRegression(fit_intercept=False)
m_clf.fit(en_en[['tree_clf', 'rf_clf']], en_en['ind'])

en_test = pd.DataFrame()

en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] =  pd.DataFrame(rf_clf.predict_proba(X_test))[1]
col_name = en_en.columns
en_test['combined'] = m_clf.predict(en_test[['tree_clf', 'rf_clf']])

col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')

tmp

['tree_clf', 'rf_clf', 'combined', 'ind']

In [12]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)
en_test.columns = tmp
print(pd.crosstab(en_test['ind'], en_test['combined']))

combined    0   1
ind              
0         275  38
1          31  24


In [13]:
print(round(accuracy_score(en_test['ind'], en_test['combined']), 4))

0.8125


In [14]:
print(classification_report(en_test['ind'], en_test['combined']))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       313
           1       0.39      0.44      0.41        55

    accuracy                           0.81       368
   macro avg       0.64      0.66      0.65       368
weighted avg       0.82      0.81      0.82       368



---
## Single Classifier

In [17]:
df = pd.read_csv("data//WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [18]:
df.Attrition.value_counts() / df.Attrition.count()

Attrition
No     0.838776
Yes    0.161224
Name: count, dtype: float64

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

class_weight = {0:0.834, 1:0.166}

pd.Series(list(y_train)).value_counts() / pd.Series(list(y_train)).count()

[0]    0.834846
[1]    0.165154
Name: count, dtype: float64

In [21]:
forest = RandomForestClassifier(class_weight=class_weight)

ada = AdaBoostClassifier(estimator=forest, n_estimators=100,
                         learning_rate=0.5, random_state=42)
ada.fit(X_train, y_train.ravel())

print_score(ada, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       920
           1       1.00      1.00      1.00       182

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[920   0]
 [  0 182]]

ROC AUC: 1.0000



In [22]:
print_score(ada, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8614

Classification Report: 
               precision    recall  f1-score   support

           0       0.87      0.98      0.92       313
           1       0.61      0.20      0.30        55

    accuracy                           0.86       368
   macro avg       0.74      0.59      0.61       368
weighted avg       0.83      0.86      0.83       368


Confusion Matrix: 
 [[306   7]
 [ 44  11]]

ROC AUC: 0.5888



In [23]:
bag_clf = BaggingClassifier(estimator=ada, n_estimators=50,
                            max_samples=1.0, max_features=1.0, bootstrap=True,
                            bootstrap_features=False, n_jobs=-1,
                            random_state=42)
bag_clf.fit(X_train, y_train.ravel())

print_score(bag_clf, X_train, X_test, y_train, y_test, train=True)

Train Result:

accuracy score: 0.9955

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       920
           1       1.00      0.97      0.99       182

    accuracy                           1.00      1102
   macro avg       1.00      0.99      0.99      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[920   0]
 [  5 177]]

ROC AUC: 0.9863



In [24]:
print_score(bag_clf, X_train, X_test, y_train, y_test, train=False)

Test Result:

accuracy score: 0.8533

Classification Report: 
               precision    recall  f1-score   support

           0       0.86      0.99      0.92       313
           1       0.56      0.09      0.16        55

    accuracy                           0.85       368
   macro avg       0.71      0.54      0.54       368
weighted avg       0.82      0.85      0.81       368


Confusion Matrix: 
 [[309   4]
 [ 50   5]]

ROC AUC: 0.5391

